In [1]:
print("Hello World!")

Hello World!


In [3]:
!nvidia-smi

Wed Jul 17 15:09:57 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA L40          On   | 00000000:9D:00.0 Off |                    0 |
| N/A   29C    P8    34W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 1.1 MB/s eta 0:00:0000:0100:020m


In [11]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import os
import cv2

In [12]:
torch.manual_seed(1)
DOWNLOAD_MNIST = True

In [13]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [01:10<00:00, 140111.40it/s]


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 30705.18it/s]


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:21<00:00, 76924.61it/s] 


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 96498.93it/s]

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



In [14]:
test_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=False
)

In [15]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001

train 50 samples with 1 channel, 28 $\times$ 28 (50,1,28,28)

use DataLoader to wrap the data, help iterate the data so that to train on batch

In [16]:
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [17]:
test_x = torch.unsqueeze(test_data.train_data, dim=1).type(torch.FloatTensor)[:2000] / 255

/opt/conda/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [18]:
test_y = test_data.test_labels[:2000]

/opt/conda/lib/python3.10/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


 用class类来建立CNN模型
 CNN流程：卷积(Conv2d)-> 激励函数(ReLU)->池化(MaxPooling)->
        卷积(Conv2d)-> 激励函数(ReLU)->池化(MaxPooling)->
        展平多维的卷积成的特征图->接入全连接层(Linear)->输出

In [36]:
class MnistCNN(nn.Module):
    def __init__(self):
        super(MnistCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [37]:
cnn = MnistCNN()

In [38]:
# test on one tensor
demo_data = torch.Tensor(1, 1, 28, 28)
cnn(demo_data)
print(cnn)

MnistCNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [39]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [42]:
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == test_y).sum().item() / float(test_y.size(0))
            print('Step: ', step, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
torch.save(cnn, 'mnist_cnn.pkl')

Step:  0 | train loss: 0.0444 | test accuracy: 0.98
Step:  50 | train loss: 0.0477 | test accuracy: 0.98
Step:  100 | train loss: 0.2179 | test accuracy: 0.98
Step:  150 | train loss: 0.1162 | test accuracy: 0.97
Step:  200 | train loss: 0.0121 | test accuracy: 0.98
Step:  250 | train loss: 0.0107 | test accuracy: 0.98
Step:  300 | train loss: 0.0292 | test accuracy: 0.99
Step:  350 | train loss: 0.0208 | test accuracy: 0.98
Step:  400 | train loss: 0.0980 | test accuracy: 0.98
Step:  450 | train loss: 0.0253 | test accuracy: 0.98
Step:  500 | train loss: 0.0113 | test accuracy: 0.99
Step:  550 | train loss: 0.0089 | test accuracy: 0.98
Step:  600 | train loss: 0.1281 | test accuracy: 0.98
Step:  650 | train loss: 0.0080 | test accuracy: 0.98
Step:  700 | train loss: 0.0346 | test accuracy: 0.98
Step:  750 | train loss: 0.0176 | test accuracy: 0.98
Step:  800 | train loss: 0.0194 | test accuracy: 0.98
Step:  850 | train loss: 0.0343 | test accuracy: 0.98
Step:  900 | train loss: 0.0050

In [44]:
cnn.eval()

MnistCNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [45]:
inputs = test_x[:32]
pred = cnn(inputs)
pred = torch.max(pred, 1)[1].data.numpy()
print('Prediction number: ', pred)

Prediction number:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1]


In [46]:
img = torchvision.utils.make_grid(inputs)
img = img.numpy().transpose(1, 2, 0)